### Table of Contents
	
* [Importing libraries and loading data](#import)
* [Additional features](#add-features)
* [Categories consolidation](#consolidation)
* [Encoding categorical features](#encoding)
* [Scaling numerical features](#scaling)
* [Train/Test split](#train-test)
* [Export train/test sets](#export)

### Importing libraries and loading data<a class="anchor" id="import"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

Loading data from our Data Wrangling notebook, we didn't make a lot of changes during our EDA.

In [2]:
census_df = pd.read_csv('..\data\cleaned data.csv')
census_df.head(2)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0


Let's look again at our data types.

In [3]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      48842 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  48842 non-null  object
 13  income          48842 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


Let's create a list of our categorical and numerical features

In [4]:
num_features = list(census_df.select_dtypes(include=('int64')).columns)
num_features.remove('income') # Removing our target variable income
print(num_features)

['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [5]:
cat_features = list(census_df.select_dtypes(include=('object')).columns)
print(cat_features)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


__The approach we'll follow for preprocessing will be the following:__
- Look to see if we can create additional features
- For the categorical features, review if we should group categories with low contribution to that feature and group as 'Other' category.
- Encode categorical features
- Scale numerical features
- Train/test split

### Additional features<a class="anchor" id="add-features"></a>

We'll create 2 additional features by bucketing age and hours per week

In [6]:
# Bucketing age by groups of 10s
age_bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']
census_df['age_range'] = pd.cut(census_df['age'], bins=age_bins, labels=age_labels)

In [7]:
# Bucketing hours worked per week
hours_worked_labels = ['1 - 20', '20 - 35', '35 - 40', '40 - 45', '45 - 50', '50 - 55', '55 - 99']
census_df['hours_worked_range'] = pd.qcut(census_df['hours-per-week'], q=11, duplicates='drop', labels=hours_worked_labels)

In [8]:
census_df.head(1)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,age_range,hours_worked_range
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,30-40,35 - 40


In [9]:
# Updating cat features list to add the 2 new features
cat_features = list(census_df.select_dtypes(include=('object','category')).columns)
print(cat_features)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'age_range', 'hours_worked_range']


### Categories consolidation<a class="anchor" id="consolidation"></a>

We can group the categories based on 2 conditions:
- A percentage threshold - any categories with less than 1% will be grouped as 'Other' category
- Exclude categories from grouping if that category was identified on the EDA as having value on the prediction (example: people with Doctorates represent a small percentage, but it's a good indicator on predicting income

__workclass__

In [10]:
census_df['workclass'].value_counts(normalize=True)

Private             0.694198
Self-emp-not-inc    0.079071
Local-gov           0.064207
Unknown             0.057307
State-gov           0.040559
Self-emp-inc        0.034704
Federal-gov         0.029319
Without-pay         0.000430
Never-worked        0.000205
Name: workclass, dtype: float64

In [11]:
# Leaving Federal-gov still as separate categories as they were an important factor in our EDA
census_df['workclass'] = census_df['workclass'].replace(['Without-pay', 'Never-worked'], 'Other')
census_df['workclass'].value_counts(normalize=True)

Private             0.694198
Self-emp-not-inc    0.079071
Local-gov           0.064207
Unknown             0.057307
State-gov           0.040559
Self-emp-inc        0.034704
Federal-gov         0.029319
Other               0.000635
Name: workclass, dtype: float64

__education__

In [12]:
census_df['education'].value_counts(normalize=True)

HS-grad         0.323164
Some-college    0.222718
Bachelors       0.164305
Masters         0.054400
Assoc-voc       0.042197
11th            0.037099
Assoc-acdm      0.032779
10th            0.028439
7th-8th         0.019553
Prof-school     0.017075
9th             0.015478
12th            0.013452
Doctorate       0.012162
5th-6th         0.010421
1st-4th         0.005057
Preschool       0.001699
Name: education, dtype: float64

In [13]:
# For education, we're going to group on 12th and under, most of those categories are under 3% 
# and didn't provide a lot of information
census_df['education'] = census_df['education'].replace(['Preschool', '1st-4th','5th-6th','7th-8th','9th','10th',\
                                                         '11th','12th'], '12th and under')
census_df['education'].value_counts(normalize=True)

HS-grad           0.323164
Some-college      0.222718
Bachelors         0.164305
12th and under    0.131199
Masters           0.054400
Assoc-voc         0.042197
Assoc-acdm        0.032779
Prof-school       0.017075
Doctorate         0.012162
Name: education, dtype: float64

__marital-status__

In [14]:
census_df['marital-status'].value_counts(normalize=True)

Married-civ-spouse       0.458192
Never-married            0.329982
Divorced                 0.135805
Separated                0.031325
Widowed                  0.031080
Married-spouse-absent    0.012858
Married-AF-spouse        0.000758
Name: marital-status, dtype: float64

In [15]:
# We can group all married into 1 category
census_df['marital-status'] = census_df['marital-status'].replace(['Married-civ-spouse', 'Married-spouse-absent',\
                                                                  'Married-AF-spouse'], 'Married')
census_df['marital-status'].value_counts(normalize=True)

Married          0.471807
Never-married    0.329982
Divorced         0.135805
Separated        0.031325
Widowed          0.031080
Name: marital-status, dtype: float64

__occupation__

In [16]:
census_df['occupation'].value_counts(normalize=True)

Prof-specialty       0.126367
Craft-repair         0.125138
Exec-managerial      0.124606
Adm-clerical         0.114881
Sales                0.112690
Other-service        0.100794
Machine-op-inspct    0.061873
Unknown              0.057512
Transport-moving     0.048217
Handlers-cleaners    0.042423
Farming-fishing      0.030507
Tech-support         0.029606
Protective-serv      0.020126
Priv-house-serv      0.004955
Armed-Forces         0.000307
Name: occupation, dtype: float64

In [17]:
census_df['occupation'] = census_df['occupation'].replace(['Armed-Forces', 'Priv-house-serv'], 'Other')
census_df['occupation'].value_counts(normalize=True)

Prof-specialty       0.126367
Craft-repair         0.125138
Exec-managerial      0.124606
Adm-clerical         0.114881
Sales                0.112690
Other-service        0.100794
Machine-op-inspct    0.061873
Unknown              0.057512
Transport-moving     0.048217
Handlers-cleaners    0.042423
Farming-fishing      0.030507
Tech-support         0.029606
Protective-serv      0.020126
Other                0.005262
Name: occupation, dtype: float64

__relationship__

In [18]:
census_df['relationship'].value_counts(normalize=True)

Husband           0.403669
Not-in-family     0.257627
Own-child         0.155215
Unmarried         0.104930
Wife              0.047725
Other-relative    0.030834
Name: relationship, dtype: float64

In [19]:
# We'll group wife and husband into Married
census_df['relationship'] = census_df['relationship'].replace(['Husband','Wife'], 'Married')
census_df['relationship'].value_counts(normalize=True)

Married           0.451394
Not-in-family     0.257627
Own-child         0.155215
Unmarried         0.104930
Other-relative    0.030834
Name: relationship, dtype: float64

We won't group __race__ and __sex__ since they have very limited categories

__native-country__

In [20]:
census_df['native-country'].value_counts(normalize=True).head(10)

United-States    0.897424
Mexico           0.019471
Unknown          0.017546
Philippines      0.006040
Germany          0.004218
Puerto-Rico      0.003767
Canada           0.003726
El-Salvador      0.003173
India            0.003092
Cuba             0.002825
Name: native-country, dtype: float64

In [21]:
# For country we'll group in US, non-US and leave the unknowns to simplify
census_df.loc[(census_df['native-country'] != 'United-States') & (census_df['native-country'] != 'Unknown')\
              ,'native-country'] = 'Non-US'
census_df['native-country'].value_counts(normalize=True).head(10)

United-States    0.897424
Non-US           0.085029
Unknown          0.017546
Name: native-country, dtype: float64

__age_range__

In [22]:
census_df['age_range'].value_counts(normalize=True)

30-40     0.262848
20-30     0.249171
40-50     0.212993
50-60     0.126981
10-20     0.074178
60-70     0.056058
70-80     0.014741
80-90     0.003030
90-100    0.000000
Name: age_range, dtype: float64

In [23]:
# We can group +70 in one category
census_df['age_range'] = census_df['age_range'].replace(['70-80','80-90','90-100'], 'Older than 70')
census_df['age_range'].value_counts(normalize=True)

30-40            0.262848
20-30            0.249171
40-50            0.212993
50-60            0.126981
10-20            0.074178
60-70            0.056058
Older than 70    0.017772
Name: age_range, dtype: float64

For __hours_worked_range__ we'll leave as is

### Encoding categorical features<a class="anchor" id="encoding"></a>

In [24]:
print(cat_features)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'age_range', 'hours_worked_range']


In [25]:
pd.set_option('display.max_columns', None) # Setting option to display all columns
census_df.shape # Getting # of features prior to encoding

(48842, 16)

In [26]:
# Creating dummy variables for categorical columns
census_df_enc = pd.get_dummies(census_df, columns=cat_features, drop_first=True, prefix=cat_features)
census_df_enc.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Local-gov,workclass_Other,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Unknown,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_Unknown,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_United-States,native-country_Unknown,age_range_20-30,age_range_30-40,age_range_40-50,age_range_50-60,age_range_60-70,age_range_Older than 70,hours_worked_range_20 - 35,hours_worked_range_35 - 40,hours_worked_range_40 - 45,hours_worked_range_45 - 50,hours_worked_range_50 - 55,hours_worked_range_55 - 99
0,39,13,2174,0,40,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,50,13,0,0,13,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,38,9,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
3,53,7,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,28,13,0,0,40,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [27]:
census_df_enc.shape

(48842, 61)

We went from 16 to 61 features total

### Scaling numerical features<a class="anchor" id="scaling"></a>

Before scaling our numerical features, we'll drop 'age' and 'hours per week' since we'll use the 2 new categorical features we created.

In [29]:
census_df_enc = census_df_enc.drop(['age','hours-per-week'], axis=1)
census_df_enc.head(1)

,education-num,capital-gain,capital-loss,income,workclass_Local-gov,workclass_Other,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Unknown,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_Unknown,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_United-States,native-country_Unknown,age_range_20-30,age_range_30-40,age_range_40-50,age_range_50-60,age_range_60-70,age_range_Older than 70,hours_worked_range_20 - 35,hours_worked_range_35 - 40,hours_worked_range_40 - 45,hours_worked_range_45 - 50,hours_worked_range_50 - 55,hours_worked_range_55 - 99
0,13,2174,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0


We'll use Robust Scaler for 'education-num' and Standard Scaler for 'capital-gain' and 'capital-loss'.

In [33]:
rob_scaler = RobustScaler()
rob_scaler.fit(census_df_enc[['education-num']])
census_df_enc[['education-num']] = rob_scaler.transform(census_df_enc[['education-num']])

In [34]:
std_scaler = StandardScaler()
std_scaler.fit(census_df_enc[['capital-gain','capital-loss']])
census_df_enc[['capital-gain','capital-loss']] = std_scaler.transform(census_df_enc[['capital-gain','capital-loss']])

In [35]:
census_df_enc.head()

,education-num,capital-gain,capital-loss,income,workclass_Local-gov,workclass_Other,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Unknown,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_Unknown,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_United-States,native-country_Unknown,age_range_20-30,age_range_30-40,age_range_40-50,age_range_50-60,age_range_60-70,age_range_Older than 70,hours_worked_range_20 - 35,hours_worked_range_35 - 40,hours_worked_range_40 - 45,hours_worked_range_45 - 50,hours_worked_range_50 - 55,hours_worked_range_55 - 99
0,1.000000,0.146932,-0.217127,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,1.000000,-0.144804,-0.217127,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,-0.333333,-0.144804,-0.217127,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
3,-1.000000,-0.144804,-0.217127,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,1.000000,-0.144804,-0.217127,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


### Train/test split<a class="anchor" id="train-test"></a>

Creating our feature matrix X and target variable y.

In [36]:
X = census_df_enc.drop(['income'], axis=1)
y = census_df_enc['income']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [38]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(34189, 58) (14653, 58) (34189,) (14653,)


### Exporting train/test sets<a class="anchor" id="export"></a>

In [39]:
# Exporting without the index
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)